# 라이브러리 불러오기

In [16]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.model_selection import KFold

import lightgbm as lgb
from catboost import CatBoostRegressor, Pool
import wandb
from wandb.integration.lightgbm import log_summary
from wandb.integration.catboost import WandbCallback
import optuna
optuna.logging.set_verbosity(optuna.logging.INFO)  # optuna log 설정

import warnings
warnings.filterwarnings('ignore')

import func.preprocessing as pp
import func.features as ft
import func.models as mdl
from func.utils import lgb_wandb_callback

# WandB 설정

In [2]:
wandb.login()
print(wandb.api.default_entity)

wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: asduianm (recsys008-naver-boostcamp). Use `wandb login --relogin` to force relogin


recsys008-naver-boostcamp


# 랜덤 시드 설정

In [3]:
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)

# 데이터 불러오기

In [4]:
path = "../../data/"  # 알잘딱 수정
file_dict = {'train':'train_data', 'test':'test_data', 
             'sample_submission':'sample_submission', 'parkInfo':'park', 
             'schoolinfo':'school', 'subwayInfo':'subway'}

for f, d in file_dict.items():
    exec(f"{d} = pp.load_data(path + '{f}.csv')")


# 중복 값 처리

In [21]:
train_data = pp.preprocess_data(train_data)

# 가장 가까운 지하철 거리 추가

In [6]:
# 각 아파트에 대해 가까운 지하철역 거리 추가
train_data = ft.calculate_nearest_subway_distance(train_data, subway)
test_data = ft.calculate_nearest_subway_distance(test_data, subway)

# 학교 레벨별로 가장 가까운 학교 추가

In [7]:
# 각 아파트에 대해 가까운 학교 거리 추가
train_data = ft.calculate_nearest_school_distance(train_data, school)
test_data = ft.calculate_nearest_school_distance(test_data, school)


# 특정 반경(radius) 내 공원 밀도

In [8]:
radius_km = 3
item_name = 'park'

# 유니크한 아파트 좌표로 공원 개수와 밀도 계산 후 결과를 원래 데이터에 매핑
train_data = ft.map_item_density_with_area(train_data, park, radius_km, item_name)
test_data = ft.map_item_density_with_area(test_data, park, radius_km, item_name)

100%|██████████| 11885/11885 [00:05<00:00, 2118.37it/s]


# 특정 거리(distance) 내 레벨별 학교 개수

In [9]:
# 각 레벨에 대해 다른 거리 범위를 설정
distance_kms = {
    'elementary': 1,  # 1km 이내
    'middle': 5,      # 3km 이내
    'high': 5         # 5km 이내
}

train_data = ft.map_school_level_counts(train_data, school, distance_kms, n_jobs=8)
test_data = ft.map_school_level_counts(test_data, school, distance_kms, n_jobs=8)

100%|██████████| 11885/11885 [00:07<00:00, 1502.89it/s]


# Holdout 데이터셋 설정 (예: 2023년 7월부터 12월까지의 데이터)

In [10]:
# 전체 재학습 데이터
all_data = train_data.copy()
X_all, y_all = pp.split_X_y(all_data, 'deposit')
X_test = test_data.copy()


In [11]:
holdout_start, holdout_end = 202307, 202312
train_data, holdout_data = pp.split_holdout_data(train_data.copy(), holdout_start, holdout_end)
X_train, y_train = pp.split_X_y(train_data, 'deposit')
X_holdout, y_holdout = pp.split_X_y(holdout_data, 'deposit')


# DBSCAN 클러스터링 
* 경도, 위도 스케일링 후 DBSCAN으로 train 기반하여 클러스터 생성
* holdout(=validation)은 경도,위도 기준 가장 가까운 train 샘플의 라벨을 할당

In [12]:
# # Train 데이터에 DBSCAN 적용
# # 클러스터 정보를 포함한 데이터셋 생성
# X_train['cluster'], X_holdout['cluster'] = ft.apply_dbscan_clustering(X_train, X_holdout)

# 모델 훈련

## 피처 선택

In [12]:
# 피처 선택
train_columns = ['area_m2', 'contract_year_month', 'floor', 'built_year', 'latitude', 'longitude',
                 'nearest_subway_distance',  'nearest_elementary_distance', 'nearest_middle_distance',
                 'nearest_high_distance', 'park_density', 'elementary', 'middle', 'high']

X_train = X_train[train_columns]
X_holdout = X_holdout[train_columns]
X_all = X_all[train_columns]
X_test = X_test[train_columns]

# LightGBM 모델 훈련

In [ ]:
lgb_params = {
    'objective': 'regression',
    'metric': ['mae', 'rmse'],
    'boosting_type': 'gbdt',
    'num_leaves': 1200,
    'min_samples': 20,
    'learning_rate': 0.035,
    'n_estimators': 2000,
    'feature_fraction': 0.65,
    'lambda_l1': 1.19,
    'lambda_l2': 4.38,
    'verbose': -1,
    'random_state': 42
}

lgb_model = mdl.train_lgb(X_train, y_train, X_holdout, y_holdout, lgb_params, 
                                 project_name="Trash can", 
                                 experiment_name="v6 + seasonal month", 
                                 entity_name="recsys008-naver-boostcamp")

## 교차검증 학습

In [ ]:
# 사용 예시
lgb_params = {
    'objective': 'regression',
    'metric': ['mae', 'rmse'],
    'boosting_type': 'gbdt',
    'num_leaves': 1200,
    'min_samples': 20,
    'learning_rate': 0.035,
    'n_estimators': 2000,
    'feature_fraction': 0.65,
    'lambda_l1': 1.19,
    'lambda_l2': 4.38,
    'verbose': -1,
    'random_state': 42
}

# 모델 훈련 및 결과
models, oof_predictions = mdl.cross_validation_lgb_with_wandb(X_all, y_all, lgb_params, 
                                                          project_name="lgbm CV", 
                                                          entity_name="recsys008-naver-boostcamp")



In [ ]:
# 테스트 데이터가 있다면 아래 코드를 사용
test_predictions = mdl.soft_voting_predict(models, X_test)
sample_submission['deposit'] = test_predictions
sample_submission.to_csv('output.csv', index=False, encoding='utf-8-sig')

## Optuna 파라미터 서칭

### 단일 학습/검증

In [ ]:
def objective(trial):

    # 하이퍼 파라미터 범위 지정
    params = {
        'metric': ['mae', 'rmse'],
        'num_leaves': trial.suggest_int('num_leaves', 1100, 1500, step=100),
        'learning_rate': trial.suggest_float('learning_rate', 0.02, 0.04, step=0.005),
        # 'n_estimators': trial.suggest_int('n_estimators', 1500, 2000, step=100),
        'n_estimators': 2000,
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 30, step=5),
        # 'bagging_fraction': trial.suggest_float('bagging_fraction', 0.6, 0.8, step=0.05),  # 각 트리가 사용할 데이터의 비율 eg. 0.8이면 80퍼센트의 데이터 샘플만 사용
        # 'bagging_freq': trial.suggest_int('bagging_freq', 0, 5),  # 몇번째 트리마다 배깅을 적용할건지 eg. 5이면 5번째 트리마다 배깅 적용
        'feature_fraction': trial.suggest_float('feature_fraction', 0.6, 1.0, step=0.05),  # 각 트리가 사용할 컬럼의 비율 eg. 0.8이면 10개의 컬럼 중 8개만 사용
        "lambda_l1": trial.suggest_float("lambda_l1", 1e-8, 10.0, log=True),
        "lambda_l2": trial.suggest_float("lambda_l2", 1e-8, 10.0, log=True),
        'num_threads': 4,
        'random_state': RANDOM_SEED,
        'verbose': 0,
    }

    # lgb model 선언 및 훈련
    # LightGBM 데이터셋 생성
    train_data = lgb.Dataset(X_train, label=y_train)
    valid_data = lgb.Dataset(X_holdout, label=y_holdout, reference=train_data)

    # LightGBM 모델 학습
    lgb_model = lgb.train(
        params,
        train_data,
        valid_sets=[valid_data],
        valid_names='validation',
        callbacks=[lgb.early_stopping(stopping_rounds=100)]
    )

    # holdout에 대한 예측
    holdout_pred = lgb_model.predict(X_holdout)
    mae = mean_absolute_error(y_holdout, holdout_pred)
    # rmse = np.sqrt(mean_squared_error(y_holdout, holdout_pred))
    # r2 = r2_score(y_holdout, holdout_pred)

    return mae


# Optuna 객체 생성
study = optuna.create_study(direction='minimize')

# MAE 최적화 수행
study.optimize(objective,
               n_trials=100,  # 몇번의 서칭을 할건지
               n_jobs=2,  # 사용할 쓰레드의 수, -1이면 최대 실제 코어 개수
)

# 최적 런과 파라미터 출력
best_params = study.best_params
print("Best parameters:", best_params)
print("Best MAE:", study.best_value)

### kfold CV

In [ ]:
def objective(trial):
    # 하이퍼파라미터 범위 지정
    params = {
        'objective': 'regression',
        'metric': ['mae', 'rmse'],
        'boosting_type': 'gbdt',
        'num_leaves': trial.suggest_int('num_leaves', 1100, 1500, step=100),
        'min_child_samples': trial.suggest_int('min_child_samples', 10, 30, step=5),
        'learning_rate': trial.suggest_float('learning_rate', 0.02, 0.05, step=0.005),
        'n_estimators': 2000,
        'feature_fraction': trial.suggest_float('feature_fraction', 0.6, 0.8, step=0.05),
        'lambda_l1': trial.suggest_float('lambda_l1', 1e-8, 10.0, log=True),
        'lambda_l2': trial.suggest_float('lambda_l2', 1e-8, 10.0, log=True),
        'random_state': RANDOM_SEED,
        'num_threads': 4,
        'verbose': 0
    }

    # 5-fold 교차 검증 준비
    kf = KFold(n_splits=5, shuffle=True, random_state=RANDOM_SEED)
    
    mae_list = []

    # 5-fold 교차 검증 수행
    for fold, (train_idx, val_idx) in enumerate(kf.split(X_all), 1):
        X_train, X_val = X_all.iloc[train_idx], X_all.iloc[val_idx]
        y_train, y_val = y_all.iloc[train_idx], y_all.iloc[val_idx]
        
        train_data = lgb.Dataset(X_train, label=y_train)
        val_data = lgb.Dataset(X_val, label=y_val, reference=train_data)
        
        model = lgb.train(
            params,
            train_data,
            valid_sets=[val_data],
            valid_names='validation',
            callbacks=[lgb.early_stopping(stopping_rounds=100)]
        )
        
        # 검증 세트에 대한 예측
        val_pred = model.predict(X_val)
        
        # MAE 계산
        mae = mean_absolute_error(y_val, val_pred)
        mae_list.append(mae)

    # 평균 MAE 반환
    return np.mean(mae_list)

# Optuna 연구 생성 및 최적화 실행
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=20, n_jobs=2)

# 최적의 하이퍼파라미터 및 성능 출력
print("Best parameters:", study.best_params)
print("Best MAE:", study.best_value)

[I 2024-10-21 16:12:52,312] A new study created in memory with name: no-name-b1302280-56b9-4759-b9a9-afecfc972b74


Training until validation scores don't improve for 100 rounds
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[1211]	validation's l1: 4343.8	validation's rmse: 7384.13
[LightGBM] [Warning] Detected that num_threads changed during training (from 4 to 8), it may cause unexpected errors.
[LightGBM] [Warning] Detected that num_threads changed during training (from 8 to 4), it may cause unexpected errors.
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[1806]	validation's l1: 4330.82	validation's rmse: 7387.05
[LightGBM] [Warning] Detected that num_threads changed during training (from 4 to 8), it may cause unexpected errors.
[LightGBM] [Warning] Detected that num_threads changed during training (from 8 to 4), it may cause unexpected errors.
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[1604]	validation's l1: 4314.88	validation's rmse: 7332.04

# Catboost 모델 훈련

In [31]:
# # wandb 초기화
# wandb.init(project="house_price_prediction", name="catboost_gpu")  # 실험명에 따라 name 변경해주기!!

# # 카테고리형 변수 정의
# cat_features = ['floor', 'built_year', 'elementary', 'middle', 'high', 'cluster']

# # CatBoost 파라미터 설정
# cat_params = {
#     'iterations': 10000,
#     'loss_function': 'RMSE',
#     'eval_metric': 'MAE',
#     'early_stopping_rounds': 100,
#     'verbose': 100,
#     'random_seed': RANDOM_SEED,
#     'task_type': 'GPU',
#     'devices': '0'
# }

# # wandb에 파라미터 로깅
# wandb.config.update(cat_params)

# # CatBoost 데이터셋 생성
# train_pool = Pool(X_train, y_train)
# valid_pool = Pool(X_holdout, y_holdout)

# # CatBoost 모델 생성
# cat_model = CatBoostRegressor(**cat_params)

# # 모델 학습
# cat_model.fit(
#     train_pool,
#     eval_set=valid_pool,
#     use_best_model=True,
#     # callbacks=[WandbCallback()]
# )

# # Holdout 데이터 예측
# cat_holdout_pred = cat_model.predict(X_holdout)

# # 성능 메트릭 계산
# cat_holdout_mae = mean_absolute_error(y_holdout, cat_holdout_pred)
# cat_holdout_rmse = np.sqrt(mean_squared_error(y_holdout, cat_holdout_pred))
# cat_holdout_r2 = r2_score(y_holdout, cat_holdout_pred)

# # wandb에 성능 지표 로깅
# wandb.log({
#     "holdout_mae": cat_holdout_mae,
#     "holdout_rmse": cat_holdout_rmse,
#     "holdout_r2": cat_holdout_r2
# })

# # 결과 출력
# print("Holdout 데이터셋 CatBoost 성능:")
# print(f"MAE: {cat_holdout_mae:.2f}")
# print(f"RMSE: {cat_holdout_rmse:.2f}")
# print(f"R²: {cat_holdout_r2:.2f}")

# # 특성 중요도 로깅
# feature_importance = cat_model.get_feature_importance()
# feature_importance_dict = dict(zip(X_train.columns, feature_importance))
# wandb.log({"feature_importance": wandb.plot.bar(wandb.Table(data=[[k, v] for k, v in feature_importance_dict.items()], columns=["feature", "importance"]), "feature", "importance", title="Feature Importance")})

# # wandb 실험 종료
# wandb.finish()

# Sample Submission 제출하기

## 전체 데이터로 클러스터링 다시하기

### DBSCAN

In [17]:
# # DBSCAN 클러스터 정보를 포함한 데이터셋 생성
# X_all['cluster'], X_test['cluster'] = ft.apply_dbscan_clustering(X_all, X_test)
# # 피처 선택
# X_all = X_all[train_columns]
# X_test = X_test[train_columns]

## 전체 데이터로 LGBM 재학습

In [ ]:
# LightGBM 파라미터 설정
lgb_params = {
    'objective': 'regression',
    'metric': 'mae',
    'boosting_type': 'gbdt',
    'num_leaves': 1200,  # 각 트리의 최대 리프 수
    'min_samples': 20,  # 각 리프의 최소 샘플 수
    'learning_rate': 0.035,
    'n_estimators': 2000,  # 트리를 몇 개 사용하여 부스팅할건지, epoch와 비슷함
    'feature_fraction': 0.65,  # 각 트리가 사용할 컬럼의 비율 eg. 0.8이면 10개의 컬럼 중 8개만 사용
    # 'bagging_fraction': 0.65,  # 각 트리가 사용할 데이터의 비율 eg. 0.8이면 80퍼센트의 데이터 샘플만 사용
    # 'bagging_freq': 0,  # 몇번째 트리마다 배깅을 적용할건지 eg. 5이면 5번째 트리마다 배깅 적용
    'lambda_l1': 1.1939606848809192,
    'lambda_l2': 4.389852271719141,
    'verbose': -1,
    'random_state': RANDOM_SEED
}


mdl.train_lgb(X_all, y_all, lgb_params, X_test=X_test, sample_submission=sample_submission)


## 전체 데이터로 Catboost 재학습

In [35]:
# # 피처 선택
# X_all = X_all[train_columns]
# X_test = X_test[train_columns]

# # CatBoost 파라미터 설정
# cat_params = {
#     'iterations': 1000,
#     'loss_function': 'RMSE',
#     'eval_metric': 'MAE',
#     'verbose': 100,
#     'random_seed': RANDOM_SEED
# }

# # CatBoost 데이터셋 생성
# all_pool = Pool(X_all, y_all)
# test_pool = Pool(X_test)

# # CatBoost 모델 생성
# cat_model = CatBoostRegressor(**cat_params)

# # 모델 학습
# cat_model.fit(all_pool)

# # 추론
# cat_test_pred = cat_model.predict(test_pool)
# sample_submission['deposit'] = cat_test_pred
# sample_submission.to_csv('output.csv', index=False, encoding='utf-8-sig')